In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import *
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from urllib.parse import urlparse

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

import time
import json
import os
import csv
import re


In [21]:
# 定义要爬取的所有URL
urls = [
    ('https://www.1976.com.tw/cat/27?t=all', '女士香水'),
    ('https://www.1976.com.tw/cat/40?t=all', '男士香水'),
    ('https://www.1976.com.tw/cat/56?t=all', '中性香水'),
]

In [22]:
def open_driver(url):
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument("--incognito")
    prefs = {
        "profile.default_content_setting_values.autofill": 2,
        "profile.password_manager_enabled": False,
        "credentials_enable_service": False,
        "autofill.profile_enabled": False,
        "autofill.address_enabled": False,
        "autofill.credit_card_enabled": False,
    }
    options.add_experimental_option("prefs", prefs)

    driver = None
    # 优先：webdriver-manager 自动匹配驱动
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
    except Exception as e1:
        fallback_path = r"chromedriver.exe"  # 如无，请修改为你的实际路径
        try:
            service = Service(fallback_path)
            driver = webdriver.Chrome(service=service, options=options)
        except Exception as e2:
            raise RuntimeError(f"无法初始化ChromeDriver：\n1) webdriver-manager错误: {e1}\n2) 本地驱动错误: {e2}")

    driver.get(url)
    driver.implicitly_wait(5)
    driver.maximize_window()
    return driver

In [ ]:
SCROLL_PAUSE = 0.5  # 列表滚动后的等待时间
DETAIL_LOAD_PAUSE = 0.5  # 打开详情页后的等待时间
BETWEEN_ITEMS_PAUSE = 0.3  # 切换商品间的间隔
BATCH_PAUSE = 1  # 每批商品后的额外休息
MAX_RETRIES = 3  # 最大重试次数

def scrape_perfume_data(driver, csv_filename="perfume_data.csv"):
    '''
    抓取香水数据的主函数，逐条写入CSV
    '''
    wait = WebDriverWait(driver, 15)
    detail_links = collect_detail_links(driver, wait)
    perfumes_data = []
    fieldnames = [
        "name",
        "original_price",
        "special_price",
        "specification",
        "description",
        "scent_notes",
        "top_notes",
        "middle_notes",
        "base_notes",
        "detail_url",
    ]

    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        print(f"共收集到 {len(detail_links)} 个商品链接需要采集")
        success_count = 0
        fail_count = 0

        for idx, detail_url in enumerate(detail_links, start=1):
            print(f"[{idx}/{len(detail_links)}] 正在处理: {detail_url}")
            
            # 添加重试机制
            retry_count = 0
            while retry_count < MAX_RETRIES:
                try:
                    perfume_info = visit_detail_page(driver, wait, detail_url)
                    if perfume_info:
                        perfumes_data.append(perfume_info)
                        writer.writerow({key: perfume_info.get(key, "") for key in fieldnames})
                        csvfile.flush()
                        success_count += 1
                        print(f"    ✓ 成功 ({success_count}/{idx})")
                    else:
                        fail_count += 1
                        print(f"    ✗ 失败 ({fail_count}/{idx})")
                    break  # 成功后跳出重试循环
                except Exception as e:
                    retry_count += 1
                    error_msg = str(e)
                    
                    # 如果是会话失效错误，尝试恢复
                    if "invalid session id" in error_msg.lower() or "no such window" in error_msg.lower():
                        print(f"    ⚠ 会话失效，尝试恢复...")
                        try:
                            # 检查是否还有窗口
                            if len(driver.window_handles) > 0:
                                driver.switch_to.window(driver.window_handles[0])
                                print(f"    ✓ 已切换到主窗口")
                            else:
                                print(f"    ✗ 无可用窗口，停止爬取")
                                return perfumes_data
                        except:
                            print(f"    ✗ 无法恢复会话，停止爬取")
                            return perfumes_data
                    
                    if retry_count < MAX_RETRIES:
                        print(f"    ⚠ 错误，第 {retry_count} 次重试: {error_msg[:50]}")
                        time.sleep(2)
                    else:
                        fail_count += 1
                        print(f"    ✗ 失败（已重试{MAX_RETRIES}次）: {error_msg[:50]}")
            
            time.sleep(BETWEEN_ITEMS_PAUSE)
            
            # 每10个商品显示进度
            if idx % 10 == 0:
                print(f"\n📊 进度报告: 已处理 {idx}/{len(detail_links)}, 成功: {success_count}, 失败: {fail_count}\n")
                time.sleep(BATCH_PAUSE)

    print(f"\n{'='*60}")
    print(f"✅ 爬取完成！")
    print(f"总链接数: {len(detail_links)}")
    print(f"成功: {success_count} 条")
    print(f"失败: {fail_count} 条")
    print(f"成功率: {success_count/len(detail_links)*100:.1f}%")
    print(f"{'='*60}")
    
    return perfumes_data

def collect_detail_links(driver, wait, max_scroll_rounds=12):
    '''
    收集商品列表中的所有详情链接，处理懒加载
    '''
    collected = []
    seen = set()
    last_height = 0

    print("开始收集商品链接...")
    
    for round_idx in range(max_scroll_rounds):
        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.item.prod-item")))
        except TimeoutException:
            print(f"第 {round_idx+1} 轮: 等待商品列表加载超时，提前结束")
            break

        items = driver.find_elements(By.CSS_SELECTOR, "div.item.prod-item")
        if not items:
            print(f"第 {round_idx+1} 轮: 未找到商品，结束")
            break
        
        before_count = len(collected)
        
        for item in items:
            try:
                # 获取详情链接
                anchor = item.find_element(By.CSS_SELECTOR, "a.inlineblock")
                href = anchor.get_attribute("href")
                
                if href and href not in seen:
                    seen.add(href)
                    collected.append(href)
            except StaleElementReferenceException:
                continue

        after_count = len(collected)
        print(f"第 {round_idx+1} 轮滚动: 新增 {after_count - before_count} 个商品，累计 {after_count} 个")
        
        driver.execute_script("window.scrollBy(0, window.innerHeight * 2);")
        time.sleep(SCROLL_PAUSE)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("页面高度未变化，已到底部")
            break
        last_height = new_height

    print(f"✓ 收集完成，共 {len(collected)} 个商品\n")
    return collected

def visit_detail_page(driver, wait, detail_url):
    '''
    打开详情页并提取数据
    '''
    main_window = None
    new_window = None
    
    try:
        # 保存主窗口句柄
        main_window = driver.current_window_handle
        
        # 打开新标签页
        driver.execute_script("window.open(arguments[0], '_blank');", detail_url)
        
        # 等待新窗口出现
        time.sleep(0.3)
        
        # 切换到新窗口
        all_windows = driver.window_handles
        for window in all_windows:
            if window != main_window:
                new_window = window
                break
        
        if new_window:
            driver.switch_to.window(new_window)
        else:
            raise Exception("无法打开新标签页")
        
        time.sleep(DETAIL_LOAD_PAUSE)
        
        # 提取数据
        perfume_info = extract_perfume_details(driver, wait)
        perfume_info["detail_url"] = detail_url
        
        return perfume_info
        
    except Exception as exc:
        print(f"    抓取详情页出错: {str(exc)[:100]}")
        raise
        
    finally:
        # 关闭新标签页并返回主窗口
        try:
            # 如果当前在新窗口，关闭它
            if new_window and driver.current_window_handle == new_window:
                driver.close()
            
            # 切换回主窗口
            if main_window:
                try:
                    driver.switch_to.window(main_window)
                except:
                    # 如果主窗口失效，尝试切换到任何可用窗口
                    if len(driver.window_handles) > 0:
                        driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            # 如果窗口管理失败，尝试切换到第一个可用窗口
            try:
                if len(driver.window_handles) > 0:
                    driver.switch_to.window(driver.window_handles[0])
            except:
                pass
        
        time.sleep(BETWEEN_ITEMS_PAUSE / 2)

def parse_fragrance_section(lines):
    '''
    解析香调及前味/中味/後味信息
    '''
    parsed = {
        "scent_notes": "未提供",
        "top_notes": "未提供",
        "middle_notes": "未提供",
        "base_notes": "未提供",
    }

    alias_map = {
        "scent_notes": ["香調", "香调"],
        "top_notes": ["前味", "前調", "前调", "TopNotes"],
        "middle_notes": ["中味", "中調", "中调", "MiddleNotes", "HeartNotes"],
        "base_notes": ["後味", "后味", "後調", "后調", "BaseNotes"],
    }

    for line in lines:
        normalized = re.sub(r"\s+", "", line)
        for key, aliases in alias_map.items():
            if any(alias in normalized for alias in aliases):
                value = line
                if "：" in line:
                    value = line.split("：", 1)[1].strip()
                elif ":" in line:
                    value = line.split(":", 1)[1].strip()
                parsed[key] = value
                break

    return parsed

def extract_perfume_details(driver, wait):
    '''
    从详情页提取香水详细信息
    '''
    info = {
        "name": "未知名称",
        "original_price": "未提供",
        "special_price": "未提供",
        "specification": "未提供",
        "description": "未提供",
        "scent_notes": "未提供",
        "top_notes": "未提供",
        "middle_notes": "未提供",
        "base_notes": "未提供",
    }

    NAME_XPATH = "/html/body/div[1]/div[2]/div[2]/div[2]/div[1]"
    DESC_XPATH = "/html/body/div[1]/div[2]/div[2]/div[2]/div[3]"
    NOTES_XPATH = "/html/body/div[1]/div[2]/div[2]/div[2]/div[4]"

    # 提取名称
    try:
        name_el = wait.until(EC.presence_of_element_located((By.XPATH, NAME_XPATH)))
        info["name"] = name_el.text.strip()
    except TimeoutException:
        print("    ⚠ 名称加载超时")
    
    # 提取表格信息（规格、正价、特价）
    try:
        # 找到所有的 product-td2 和 product-td3 元素
        spec_elements = driver.find_elements(By.CSS_SELECTOR, "td.product-td2")
        special_price_elements = driver.find_elements(By.CSS_SELECTOR, "td.product-td3 span.product-price")
        
        # 第一个 product-td2 是规格
        if len(spec_elements) >= 1:
            spec_text = spec_elements[0].text.strip()
            if spec_text and not spec_text.replace(',', '').replace(' ', '').isdigit():
                info["specification"] = spec_text
        
        # 第二个 product-td2 是正价
        if len(spec_elements) >= 2:
            original_price_text = spec_elements[1].text.strip()
            if original_price_text:
                info["original_price"] = original_price_text
        
        # product-td3 里的 product-price 是特价
        if special_price_elements:
            special_price_text = special_price_elements[0].text.strip()
            if special_price_text:
                info["special_price"] = special_price_text
    except Exception as e:
        print(f"    ⚠ 提取价格/规格出错: {str(e)[:50]}")

    # 提取描述
    try:
        desc_el = driver.find_element(By.XPATH, DESC_XPATH)
        desc_text = desc_el.text.strip()
        if desc_text:
            info["description"] = desc_text
    except Exception:
        pass

    # 提取香调信息
    try:
        notes_el = driver.find_element(By.XPATH, NOTES_XPATH)
        notes_lines = [line.strip() for line in notes_el.text.splitlines() if line.strip()]
        info.update(parse_fragrance_section(notes_lines))
    except Exception:
        pass

    return info

def save_to_csv(data, filename="perfume_data.csv"):
    '''
    将数据保存到CSV文件
    '''
    if not data:
        print("没有数据可保存")
        return

    keys = data[0].keys()

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)

    print(f"数据已保存至 {filename}")

def scrape_category(url, category_name, csv_filename):
    '''
    爬取单个类别的香水数据
    '''
    driver = open_driver(url)
    
    try:
        print("="*60)
        print(f"🌸 开始爬取 {category_name}")
        print("="*60 + "\n")
        
        perfumes_data = scrape_perfume_data(driver, csv_filename)
        
        return perfumes_data
        
    finally:
        print(f"\n关闭 {category_name} 的浏览器...")
        driver.quit()
        print(f"✓ {category_name} 浏览器已关闭\n")

def main():
    '''
    主函数 - 循环爬取所有类别
    '''
    all_data = []
    
    try:
        print("\n" + "="*60)
        print("🚀 开始爬取所有类别的香水数据")
        print("="*60 + "\n")
        
        for idx, (url, category_name) in enumerate(urls, 1):
            print(f"\n{'#'*60}")
            print(f"# 类别 {idx}/{len(urls)}: {category_name}")
            print(f"# URL: {url}")
            print(f"{'#'*60}\n")
            
            # 为每个类别生成独立的CSV文件名
            csv_filename = f"perfume_data_{category_name}.csv"
            
            try:
                category_data = scrape_category(url, category_name, csv_filename)
                all_data.extend(category_data)
                
                print(f"\n✅ {category_name} 爬取完成！共 {len(category_data)} 条数据")
                print(f"   数据已保存至: {csv_filename}\n")
                
                # 类别之间休息一下
                if idx < len(urls):
                    print("⏳ 休息5秒后继续下一个类别...\n")
                    time.sleep(5)
                    
            except Exception as e:
                print(f"\n❌ {category_name} 爬取失败: {str(e)}\n")
                continue
        
        # 汇总所有数据
        if all_data:
            all_csv_filename = "perfume_data_all.csv"
            print("\n" + "="*60)
            print(f"📊 汇总所有数据到: {all_csv_filename}")
            print("="*60)
            
            fieldnames = [
                "name", "original_price", "special_price", "specification", "description", 
                "scent_notes", "top_notes", "middle_notes", "base_notes", "detail_url"
            ]
            
            with open(all_csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(all_data)
            
            print(f"✓ 汇总完成！总共 {len(all_data)} 条数据")
            
            # 打印统计信息
            print(f"\n{'='*60}")
            print("📈 最终统计:")
            print("="*60)
            for url, category_name in urls:
                count = sum(1 for d in all_data if category_name in csv_filename)
                print(f"  {category_name}: 约 {len([d for d in all_data])/len(urls):.0f} 条")
            print(f"  总计: {len(all_data)} 条")
            print("="*60)
            
            # 打印前3个结果作为示例
            print(f"\n{'='*60}")
            print("📝 数据示例（前3条）:")
            print("="*60)
            for i, perfume in enumerate(all_data[:3], 1):
                print(f"\n香水 {i}:")
                for key, value in perfume.items():
                    if len(str(value)) > 50:
                        print(f"  {key}: {str(value)[:50]}...")
                    else:
                        print(f"  {key}: {value}")

    except KeyboardInterrupt:
        print("\n\n⚠ 用户中断程序")
    except Exception as e:
        print(f"\n\n❌ 执行过程中出现错误: {str(e)}")
        import traceback
        traceback.print_exc()
    finally:
        print("\n" + "="*60)
        print("✅ 所有任务完成！")
        print("="*60)


SyntaxError: unterminated string literal (detected at line 439) (1229433207.py, line 439)

In [ ]:
# 运行爬虫
if __name__ == "__main__":
    main()


🚀 开始爬取所有类别的香水数据


############################################################
# 类别 1/3: 女士香水
# URL: https://www.1976.com.tw/cat/27?t=all
############################################################

🌸 开始爬取 女士香水

开始收集商品链接...
第 1 轮滚动: 新增 1079 个商品，累计 1079 个
第 2 轮滚动: 新增 0 个商品，累计 1079 个
页面高度未变化，已到底部
✓ 收集完成，共 1079 个商品

共收集到 1079 个商品链接需要采集
[1/1079] 正在处理: https://www.1976.com.tw/prod/23311
    ✓ 成功 (1/1)
[2/1079] 正在处理: https://www.1976.com.tw/prod/23306
    ✓ 成功 (2/2)
[3/1079] 正在处理: https://www.1976.com.tw/prod/23274
    ✓ 成功 (3/3)
[4/1079] 正在处理: https://www.1976.com.tw/prod/23246
    ✓ 成功 (4/4)
[5/1079] 正在处理: https://www.1976.com.tw/prod/23236
    ✓ 成功 (5/5)
[6/1079] 正在处理: https://www.1976.com.tw/prod/23233
    ✓ 成功 (6/6)
[7/1079] 正在处理: https://www.1976.com.tw/prod/23192
    ✓ 成功 (7/7)
[8/1079] 正在处理: https://www.1976.com.tw/prod/23158
    ✓ 成功 (8/8)
[9/1079] 正在处理: https://www.1976.com.tw/prod/23138
    ✓ 成功 (9/9)
[10/1079] 正在处理: https://www.1976.com.tw/prod/23136
    ✓ 成功 (10/10)

📊 进度报告: 已